In [4]:
# ! python ../input/mlcomp/mlcomp/mlcomp/setup.py

In [5]:
import os
import json
import gc

import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [9]:
import numpy as np
import cv2
import albumentations as A
from tqdm import tqdm_notebook
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.jit import load

from mlcomp.contrib.transform.albumentations import ChannelTranspose
from mlcomp.contrib.dataset.classify import ImageDataset
from mlcomp.contrib.transform.rle import rle2mask, mask2rle
from mlcomp.contrib.transform.tta import TtaWrap

ModuleNotFoundError: No module named 'mlcomp'

In [4]:
unet_se_resnext50_32x4d = \
    load('../save_model/unet_se_resnext50_32x4d.pth').cuda()
unet_mobilenet2 = load('../save_model/unet_mobilenet2.pth').cuda()
unet_resnet34 = load('../save_model/unet_resnet34.pth').cuda()

# unet_se_resnext50_32x4d = \
#     load('../input/severstal-mlcomp-catalyst-train-0-90672-offline/unet_se_resnext50_32x4d.pth')
# unet_mobilenet2 = load('../input/severstal-mlcomp-catalyst-train-0-90672-offline/unet_mobilenet2.pth')
# unet_resnet34 = load('../input/severstal-mlcomp-catalyst-train-0-90672-offline/unet_resnet34.pth')

AssertionError: Torch not compiled with CUDA enabled

# Preprocessing

In [5]:
# os.listdir('../input/severstal-mlcomp-catalyst-train-0-90672-offline')

In [6]:
train_df = pd.read_csv('../input/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

(50272, 5)


,ImageId_ClassId,EncodedPixels,ImageId,ClassId,hasMask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,NaN,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,NaN,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,NaN,0002cc93b.jpg,4,False
4,00031f466.jpg_1,NaN,00031f466.jpg,1,False


In [8]:
sub_df = pd.read_csv('../input/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])
test_imgs.head()

,ImageId
0,004f40c73.jpg
1,006f39c41.jpg
2,00b7fb703.jpg
3,00bbcd9af.jpg
4,0108ce457.jpg


# Step 1: Remove test images without defects

Most of the stuff below is hidden, since it's copied from my previous kernels.

In [10]:
def load_img(code, base, resize=True):
    path = f'{base}/{code}'
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, (256, 256))
    return img

def validate_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [11]:
BATCH_SIZE = 64
def create_test_gen():
    return ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        test_imgs,
        directory='../input/test_images',
        x_col='ImageId',
        class_mode=None,
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

test_gen = create_test_gen()

Found 1801 validated image filenames.


In [12]:
remove_model = load_model('../input/save_model/remove_model.h5')
remove_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [13]:
test_missing_pred = remove_model.predict_generator(
    test_gen,
    steps=len(test_gen),
    verbose=1
)

test_imgs['allMissing'] = test_missing_pred
test_imgs.head()

29/29 [==============================] - 463s 16s/step


,ImageId,allMissing
0,004f40c73.jpg,0.535309
1,006f39c41.jpg,0.999956
2,00b7fb703.jpg,0.997788
3,00bbcd9af.jpg,0.000324
4,0108ce457.jpg,0.114875


In [14]:
threshold = .9996
filtered_test_imgs = test_imgs[test_imgs['allMissing'] < threshold]
print(filtered_test_imgs.shape)
filtered_test_imgs.head()

(1678, 2)


,ImageId,allMissing
0,004f40c73.jpg,0.535309
2,00b7fb703.jpg,0.997788
3,00bbcd9af.jpg,0.000324
4,0108ce457.jpg,0.114875
5,0109b68ec.jpg,0.766567


`filtered_sub_df` contains all of the images with at least one mask. `null_sub_df` contains all the images with exactly 4 missing masks.

In [15]:
filtered_mask = sub_df['ImageId'].isin(filtered_test_imgs["ImageId"].values)
filtered_sub_df = sub_df[filtered_mask].copy()
null_sub_df = sub_df[~filtered_mask].copy()
null_sub_df['EncodedPixels'] = null_sub_df['EncodedPixels'].apply(lambda x: ' ')

filtered_sub_df.reset_index(drop=True, inplace=True)
filtered_test_imgs.reset_index(drop=True, inplace=True)

print(null_sub_df.shape)

(6712, 3)
(492, 3)


# step 2: severstal-mlcomp-catalyst-infer-0-90672

In [16]:
class Model:
    def __init__(self, models):
        self.models = models
    
    def __call__(self, x):
        res = []
        x = x.cuda()
        with torch.no_grad():
            for m in self.models:
                res.append(m(x))
        res = torch.stack(res)
        return torch.mean(res, dim=0)

model = Model([unet_se_resnext50_32x4d, unet_mobilenet2, unet_resnet34])

def create_transforms(additional):
    res = list(additional)
    # add necessary transformations
    res.extend([
        A.Normalize(
            mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        ),
        ChannelTranspose()
    ])
    res = A.Compose(res)
    return res

img_folder = '/kaggle/input/test_images'
batch_size = 2
num_workers = 0

# Different transforms for TTA wrapper
transforms = [
    [],
    [A.HorizontalFlip(p=1)]
]

transforms = [create_transforms(t) for t in transforms]
datasets = [TtaWrap(ImageDataset(img_folder=img_folder, transforms=t), tfms=t) for t in transforms]
loaders = [DataLoader(d, num_workers=num_workers, batch_size=batch_size, shuffle=False) for d in datasets]

NameError: name 'unet_se_resnext50_32x4d' is not defined

In [17]:
thresholds = [0.5, 0.5, 0.5, 0.5]
min_area = [600, 600, 1000, 2000]

res = []
# Iterate over all TTA loaders
total = len(datasets[0])//batch_size
for loaders_batch in tqdm_notebook(zip(*loaders), total=total):
    preds = []
    image_file = []
    for i, batch in enumerate(loaders_batch):
#         features = batch['features']
        features = batch['features'].cuda()
        p = torch.sigmoid(model(features))
        # inverse operations for TTA
        p = datasets[i].inverse(p)
        preds.append(p)
        image_file = batch['image_file']
    
    # TTA mean
    preds = torch.stack(preds)
    preds = torch.mean(preds, dim=0)
    preds = preds.detach().cpu().numpy()
    
    # Batch post processing
    for p, file in zip(preds, image_file):
        file = os.path.basename(file)
        # Image postprocessing
        for i in range(4):
            p_channel = p[i]
            imageid_classid = file+'_'+str(i+1)
            p_channel = (p_channel>thresholds[i]).astype(np.uint8)
            if p_channel.sum() < min_area[i]:
                p_channel = np.zeros(p_channel.shape, dtype=p_channel.dtype)
            
            res.append({
                'ImageId_ClassId': imageid_classid,
                'EncodedPixels': mask2rle(p_channel)
            })

NameError: name 'datasets' is not defined

ensemble

In [18]:
df = pd.DataFrame(res)

In [19]:
null_sub_df.shape

(492, 3)

In [20]:
print((df.loc[df['ImageId_ClassId'].isin(list(null_sub_df["ImageId_ClassId"])),'EncodedPixels'] != '').sum() )

KeyError: 'ImageId_ClassId'

In [21]:
df.loc[df['ImageId_ClassId'].isin(list(null_sub_df["ImageId_ClassId"])), 'EncodedPixels'] = ''

KeyError: 'ImageId_ClassId'

In [22]:
df.to_csv('submission.csv', index=False)